# Automatizando a espacialização dos atendimentos do SP156

Agora que já exploramos os dados, vamos começar a automatizar a espacialização dos dados.

A estratégia vai ser separar por registros que satisfaçam as seguintes condições:

* pontos com Lat/Long
* Distrito e não tenham informação de Setor Fiscal ou Quadra
* Setor fiscal que não tenham informação de Quadra
* Setor e Quadra fiscal

Depois disso podemos agrupar registros por Serviço, Assunto ou Tema conforme a melhor discretização.


In [77]:
import pandas as pd
import glob

# path = r'download' # use your path
# all_files = glob.glob(path + "/*.csv")

# li = []

# for filename in all_files:
#     df = pd.read_csv(filename, index_col=None, header=0, encoding="ISO-8859-1", delimiter=';', parse_dates=['Data de abertura', 'Data do parecer'])
#     li.append(df)

# frame = pd.concat(li, axis=0, ignore_index=True)

# Abrindo o ano de 2019
df = pd.read_csv('download/dados-do-sp156---1o-sem-2019.csv',index_col=None, header=0, encoding="ISO-8859-1", delimiter=';', parse_dates=['Data de abertura', 'Data do parecer'])
df.append(pd.read_csv('download/dados-do-sp156---2o-sem-2019.csv',index_col=None, header=0, encoding="ISO-8859-1", delimiter=';', parse_dates=['Data de abertura', 'Data do parecer']))

/home/fernando/miniconda3/envs/geo_env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (6,7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Data de abertura,Canal,Tema,Assunto,Serviço,Logradouro,Número,CEP,Subprefeitura,Distrito,Setor,Quadra,Latitude,Longitude,Data do parecer,Status da solicitação,Órgão
0,2019-01-01 00:29:43,CENTRAL TELEFÔNICA,Cidadania e assistência social,População ou pessoa em situação de rua,Serviço Especializado de Abordagem Social às P...,NaN,NaN,NaN,BUTANTA,VILA SONIA,NaN,NaN,NaN,NaN,2019-01-01 04:44:28,FINALIZADA,SMADS
1,2019-01-01 00:32:04,CENTRAL TELEFÔNICA,Cidadania e assistência social,População ou pessoa em situação de rua,Serviço Especializado de Abordagem Social às P...,NaN,NaN,NaN,CAMPO LIMPO,CAMPO LIMPO,NaN,NaN,NaN,NaN,2019-01-01 04:44:28,FINALIZADA,SMADS
2,2019-01-01 00:35:47,CENTRAL TELEFÔNICA,Cidadania e assistência social,População ou pessoa em situação de rua,Serviço Especializado de Abordagem Social às P...,NaN,NaN,NaN,CAMPO LIMPO,CAMPO LIMPO,NaN,NaN,NaN,NaN,2019-01-01 04:44:28,FINALIZADA,SMADS
3,2019-01-01 00:38:05,CENTRAL TELEFÔNICA,Cidadania e assistência social,População ou pessoa em situação de rua,Serviço Especializado de Abordagem Social às P...,NaN,NaN,NaN,ERMELINO MATARAZZO,ERMELINO MATARAZZO,NaN,NaN,NaN,NaN,2019-01-01 05:30:50,FINALIZADA,SMADS
4,2019-01-01 00:43:45,CENTRAL TELEFÔNICA,Cidadania e assistência social,População ou pessoa em situação de rua,Serviço Especializado de Abordagem Social às P...,NaN,NaN,NaN,MOOCA,BRAS,NaN,NaN,NaN,NaN,2019-01-01 04:44:28,FINALIZADA,SMADS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
536408,2019-12-31 21:04:00,CENTRAL TELEFÔNICA,Lixo e limpeza,Coleta de lixo domiciliar,Reclamações sobre coleta de lixo domiciliar,Rua Toutinegra,NaN,NaN,PENHA,PENHA,59.0,NaN,NaN,NaN,2020-04-01 10:04:00,INDEFERIDO,AMLURB
536409,2019-12-31 21:06:00,CENTRAL TELEFÔNICA,Cidadania e assistência social,População ou pessoa em situação de rua,Serviço Especializado de Abordagem Social às P...,NaN,NaN,NaN,LAPA,BARRA FUNDA,NaN,NaN,NaN,NaN,2019-12-31 21:47:00,FINALIZADA,SMADS
536410,2019-12-31 21:06:00,CENTRAL TELEFÔNICA,Cidadania e assistência social,População ou pessoa em situação de rua,Serviço Especializado de Abordagem Social às P...,NaN,NaN,NaN,SANTANA-TUCURUVI,SANTANA,NaN,NaN,NaN,NaN,2019-12-31 21:46:00,FINALIZADA,SMADS
536411,2019-12-31 21:07:00,PORTAL ONLINE,Trânsito e Transporte,Estacionamento e Zona Azul,Cartão de estacionamento idoso,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-02-01 11:16:00,FINALIZADA,SMT-DSV


In [78]:
df['Latitude'] = pd.to_numeric(df['Latitude'].str.replace(',', '.'), errors='coerce')
df['Longitude'] = pd.to_numeric(df['Longitude'].str.replace(',', '.'), errors='coerce')

In [79]:
df.dtypes


Data de abertura         datetime64[ns]
Canal                            object
Tema                             object
Assunto                          object
Serviço                          object
Logradouro                       object
Número                          float64
CEP                             float64
Subprefeitura                    object
Distrito                         object
Setor                           float64
Quadra                          float64
Latitude                        float64
Longitude                       float64
Data do parecer          datetime64[ns]
Status da solicitação            object
Órgão                            object
dtype: object

In [80]:
df_points = df[~df.Longitude.isna()]

In [81]:
df_distrito = df[~df.Distrito.isna() & df.Latitude.isna() & df.Setor.isna()]

In [82]:
df_setor = df[df.Latitude.isna() & ~df.Setor.isna() & df.Quadra.isna()]

In [83]:
df_quadra = df[df.Latitude.isna() & ~df.Setor.isna() & ~df.Quadra.isna()]

In [84]:
df_points.Serviço.unique()

array(['Tapa-buraco',
       'Ônibus - Reclamação de intervalo excessivo da linha',
       'Remoção de entulho em via pública', 'Conserto de lixeira',
       'Lavagem especial de locais públicos',
       'Remoção de grandes objetos em vias públicas',
       'Remoção de animal morto em via pública',
       'Ausência de coleta do contêiner para coleta seletiva',
       'Reclamação de má execução do serviço de tapa-buraco',
       'Instalação e remoção de lixeira',
       'Limpeza de bueiros, boca de lobo e poços de visita',
       'Manutenção de placas de trânsito',
       'Capinação em guias e sarjetas', 'Reclamação de falta de varrição',
       'Limpeza da via pública após enchentes ou eventos',
       'Tapa-buraco em faixa exclusiva ou corredor de ônibus',
       'Ônibus - Denúncia de direção inadequada ou perigosa de veículo',
       'Cata Bagulho - Reclamação de não retirada de material',
       'Pintura de guias e postes',
       'Coleta seletiva - Solicitar manutenção de contêiner

In [85]:
df_distrito.Assunto.unique()

array(['População ou pessoa em situação de rua',
       'Drenagem de água de chuva', 'Reclamação ônibus', 'Defesa Civil',
       'Árvore', 'Poluição Sonora - PSIU',
       'Estabelecimentos comerciais, indústrias e serviços',
       'Criança e adolescente', 'Fiscalização de obras',
       'Locais com lotação superior a 250 pessoas (cinemas, teatros, casas de shows)',
       'Estacionamento e Zona Azul', 'Veículos abandonados',
       'Capinação e roçada de áreas verdes',
       'Animais que podem causar doenças e/ou agravos à saúde', 'Eventos',
       'Criação inadequada de animais', 'Ouvidoria Geral do Município',
       'Calçadas, guias e postes', 'Ônibus', 'Terrenos e imóveis',
       'Coleta de lixo domiciliar',
       'Ponto viciado, entulho e caçamba de entulho',
       'Dengue/chikungunya/zika (mosquito aedes aegypti)',
       'Rios e Córregos',
       'Sinalização e Circulação de Veículos e Pedestres', 'Carga',
       'Reparos em asfalto, pontes e viadutos', 'Atendimento - SF',

In [86]:
df_setor.Serviço.unique()

array(['Reclamações sobre coleta de lixo domiciliar',
       'Denúncia de má conduta dos funcionários de limpeza',
       'Reclamações sobre coleta seletiva',
       'Reclamações sobre coleta de resíduos de serviços de saúde',
       'Denúncia de má conduta dos funcionários (motoristas /coletores)',
       'Remoção de propaganda irregular'], dtype=object)

In [87]:
df_quadra.Serviço.unique()

array(['Animal encontrado com registro geral do animal (RGA)',
       'Vistoria de colmeia/vespeiro instalado',
       'Avaliação de local com pernilongo/mosquito',
       'Avaliação de animal agressor em vias públicas',
       'Avaliação de animal em sofrimento sem proprietário para eutanásia (morte sem dor)',
       'Remover animal de grande porte solto',
       'Reclamação de local com morcego',
       'Implantação de coleta porta-porta'], dtype=object)

In [88]:
# TODO
# Separar por ano
# FAzer um layer com todos os Distritos concentrados

In [89]:
df.Tema.unique()

array(['Cidadania e assistência social', 'Rua e bairro',
       'Trânsito e Transporte', 'Urgência', 'Lixo e limpeza',
       'Comércio e serviços', 'Obras, imóveis e habitação', 'Animais',
       'Ouvidoria Geral do Município', 'Bilhete Único', 'Saúde',
       'Canais de Atendimento', 'Meio Ambiente', 'SAC/GRC', 'Educação',
       'Trabalho', 'Finanças', 'Segurança urbana',
       'Cultura, esporte e lazer', 'Acessibilidade'], dtype=object)

## Pontos

No caso dos pontos podemos gerar um arquivo sem discretização, ou seja, cada registro conterá seu ponto.

Vamos utilizar uma estratégia para agrupa-los por `Serviço` dessa forma:

In [115]:
len(df_points.Serviço.unique())

37

In [116]:
import geopandas as gpd

In [117]:
gdf_points = gpd.GeoDataFrame(df_points, geometry=gpd.points_from_xy(df_points.Longitude, df_points.Latitude))

In [118]:
gdf_points.crs = 'epsg:4989'
gdf_points.to_crs('epsg:31983')

,Data de abertura,Canal,Tema,Assunto,Serviço,Logradouro,Número,CEP,Subprefeitura,Distrito,Setor,Quadra,Latitude,Longitude,Data do parecer,Status da solicitação,Órgão,geometry
22,2019-01-01 02:51:02,PORTAL ONLINE,Rua e bairro,"Reparos em asfalto, pontes e viadutos",Tapa-buraco,Rua Leite de Morais,76.0,2034020.0,SANTANA-TUCURUVI,SANTANA,73.0,38.0,-23.503278,-46.625702,2019-02-05 14:41:55,INDEFERIDO,SMSUB,POINT (334004.327 7399826.369)
43,2019-01-01 06:16:08,PORTAL ONLINE,Rua e bairro,"Reparos em asfalto, pontes e viadutos",Tapa-buraco,Rua Gaspar Lourenço,434.0,4107001.0,VILA MARIANA,VILA MARIANA,38.0,86.0,-23.582634,-46.632824,2019-04-02 18:46:41,FINALIZADA,SMSUB,POINT (333377.045 7391030.456)
44,2019-01-01 06:17:10,CENTRAL TELEFÔNICA,Trânsito e Transporte,Reclamação ônibus,Ônibus - Reclamação de intervalo excessivo da ...,Rua Guamiranga,1160.0,4220020.0,VILA PRUDENTE,VILA PRUDENTE,302.0,51.0,-23.596709,-46.584591,2019-01-02 18:50:23,FINALIZADA,SPTRANS,POINT (338317.191 7389527.124)
45,2019-01-01 06:21:31,CENTRAL TELEFÔNICA,Trânsito e Transporte,Reclamação ônibus,Ônibus - Reclamação de intervalo excessivo da ...,Avenida Professor Mario Mazagão,622.0,4929170.0,M'BOI MIRIM,JARDIM ANGELA,NaN,NaN,-23.700044,-46.764398,2019-03-13 04:10:04,FINALIZADA,SPTRANS,POINT (320108.325 7377868.545)
47,2019-01-01 06:31:20,PORTAL ONLINE,Lixo e limpeza,"Ponto viciado, entulho e caçamba de entulho",Remoção de entulho em via pública,Avenida das Nações Unidas,19659.0,4730090.0,SANTO AMARO,SANTO AMARO,NaN,NaN,-23.640986,-46.723556,2019-01-02 16:46:56,FINALIZADA,AMLURB,POINT (324194.222 7384459.843)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590166,2019-06-30 22:50:22,PORTAL ONLINE,Rua e bairro,"Reparos em asfalto, pontes e viadutos",Tapa-buraco,Rua Bento Gonçalves,235.0,3334000.0,MOOCA,AGUA RASA,NaN,NaN,-23.553722,-46.571100,2019-07-24 11:35:35,FINALIZADA,SMSUB,POINT (339641.736 7394302.494)
590170,2019-06-30 22:53:42,CENTRAL TELEFÔNICA,Lixo e limpeza,"Ponto viciado, entulho e caçamba de entulho",Remoção de entulho em via pública,Rua Professor José Lourenço,915.0,2977020.0,PIRITUBA-JARAGUA,PIRITUBA,NaN,NaN,-23.467568,-46.713742,2019-07-03 11:36:28,FINALIZADA,AMLURB,POINT (324965.907 7403676.356)
590180,2019-06-30 23:05:23,PORTAL ONLINE,Lixo e limpeza,"Ponto viciado, entulho e caçamba de entulho",Remoção de entulho em via pública,Viaduto Dante Delmanto,1126.0,4310100.0,JABAQUARA,JABAQUARA,NaN,NaN,-23.628850,-46.641316,2019-07-05 12:56:13,FINALIZADA,AMLURB,POINT (332569.005 7385902.528)
590200,2019-06-30 23:40:31,CENTRAL TELEFÔNICA,Lixo e limpeza,Remoção de grandes objetos,Remoção de grandes objetos em vias públicas,Rua Jovino Mouzinho de Pontes,141.0,5843330.0,M'BOI MIRIM,JARDIM SAO LUIS,NaN,NaN,-23.651227,-46.740997,2019-07-01 17:36:27,FINALIZADA,AMLURB,POINT (322428.694 7383304.115)


In [119]:
for s in df_points.Serviço.unique():
    print(f"{s}: ({len(df_points[df_points.Serviço == s])})")
    gdf_points[df_points.Serviço == s].to_file('resultados/Atendimentos-156-2019.gpkg', layer=f"Pontos - {s}", driver='GPKG')

Tapa-buraco: (78979)
Ônibus - Reclamação de intervalo excessivo da linha: (7220)
Remoção de entulho em via pública: (24342)
Conserto de lixeira: (339)
Lavagem especial de locais públicos: (740)
Remoção de grandes objetos em vias públicas: (24319)
Remoção de animal morto em via pública: (2070)
Ausência de coleta do contêiner para coleta seletiva: (60)
Reclamação de má execução do serviço de tapa-buraco: (960)
Instalação e remoção de lixeira: (4382)
Limpeza de bueiros, boca de lobo e poços de visita: (11610)
Manutenção de placas de trânsito: (1198)
Capinação em guias e sarjetas: (7439)
Reclamação de falta de varrição: (6804)
Limpeza da via pública após enchentes ou eventos: (1322)
Tapa-buraco em faixa exclusiva ou corredor de ônibus: (1343)
Ônibus - Denúncia de direção inadequada ou perigosa de veículo: (2196)
Cata Bagulho - Reclamação de não retirada de material: (1704)
Pintura de guias e postes: (248)
Coleta seletiva - Solicitar manutenção de contêiner: (58)
Reclamação de não remoção d

## Agrupando total de atendimentos por distrito

Parece fazer sentido ter um agrupamento geral discretizado por distrito. A princípio separado por número de atendimentos e tema. Para isso vamos ter que relacionar os registros de atendimento às feiçõe de distrito.

Porém precisamos agrupar os atendimentos por distrito.

In [107]:
len(df.Distrito.unique())

97

In [108]:
total_de_atendimentos_por_distrito = (df[['Distrito','Data de abertura']]
                                      .groupby('Distrito')
                                      .count()
                                      .rename(columns={'Data de abertura':'Total de Atendimentos'}))

In [109]:
canais_e_temas_por_distrito = (pd.get_dummies(df[['Distrito', 'Canal', 'Tema']], columns=['Canal', 'Tema'])
                               .groupby('Distrito')
                               .sum())

In [110]:
df_total_por_distrito = pd.concat([total_de_atendimentos_por_distrito, canais_e_temas_por_distrito], axis=1)

In [111]:
df_distritos = gpd.read_file('arquivos/SIRGAS_SHP_distrito_polygon.shp')

In [112]:
df_distritos_e_total_por_distrito = df_distritos.merge(df_total_por_distrito, left_on='ds_nome', right_on='Distrito')

In [113]:
df_distritos_e_total_por_distrito.to_file('resultados/Atendimentos-156-2019.gpkg', layer='Distritos - Quadro geral por canal e tema', driver='GPKG')

## Assuntos por distrito

Agora podemos começar a gerar as uma camada para cada Assunto discretizado por distrito. Ou seja, vamos separar por assuntos os registros de atendimentos que possuem somente informação do distrito

In [114]:
for a in df_distrito.Assunto.unique():
    print(f"{a}: ({len(df_distrito[df_distrito.Assunto == a])})")
    c = (df_distrito[df_distrito.Assunto == a][['Distrito','Data de abertura']]
         .groupby('Distrito')
         .count()
         .rename(columns={'Data de abertura':'Total de Atendimentos'}))
    r = df_distritos.merge(c, left_on='ds_nome', right_on='Distrito')
    r.to_file('resultados/Atendimentos-156-2019.gpkg', layer=f'Distritos por assunto - {a}', driver='GPKG')

População ou pessoa em situação de rua: (57673)
Drenagem de água de chuva: (11206)
Reclamação ônibus: (18119)
Defesa Civil: (22491)
Árvore: (39809)
Poluição Sonora - PSIU: (9449)
Estabelecimentos comerciais, indústrias e serviços: (6156)
Criança e adolescente: (278)
Fiscalização de obras: (7358)
Locais com lotação superior a 250 pessoas (cinemas, teatros, casas de shows): (374)
Estacionamento e Zona Azul: (3228)
Veículos abandonados: (16749)
Capinação e roçada de áreas verdes: (10839)
Animais que podem causar doenças e/ou agravos à saúde: (6108)
Eventos: (79)
Criação inadequada de animais: (3008)
Ouvidoria Geral do Município: (13638)
Calçadas, guias e postes: (9823)
Ônibus: (6580)
Terrenos e imóveis: (7315)
Coleta de lixo domiciliar: (1375)
Ponto viciado, entulho e caçamba de entulho: (6223)
Dengue/chikungunya/zika (mosquito aedes aegypti): (8641)
Rios e Córregos: (2214)
Sinalização e Circulação de Veículos e Pedestres: (5054)
Carga: (20)
Reparos em asfalto, pontes e viadutos: (1276)
A

## Serviços por Setor Fiscal e Setor e Quadra fiscal

Analogamente o que fizemos com os distritos, agora vamos relacionar os registros que tenham apenas Setor fiscal, assim com Setor e Quadra fiscal, discretizando os registros nas feições.

In [125]:
zipfile = "zip:///home/fernando/dev/geo-sp156/arquivos/SIRGAS_SHP_setorfiscal.zip!SIRGAS_SHP_setorfiscal/SIRGAS_SHP_setorfiscal.shp"
df_setores = gpd.read_file(zipfile)

In [147]:
# df_setores

In [148]:
df_setor['st_codigo'] = df_setor.Setor.astype(int).apply(lambda x: '{:03d}'.format(x))

<ipython-input-148-b4f69922e5aa>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_setor['st_codigo'] = df_setor.Setor.astype(int).apply(lambda x: '{:03d}'.format(x))


In [154]:
# df_setor

In [153]:
for s in df_setor.Serviço.unique():
    print(f"{s}: ({len(df_setor[df_setor.Serviço == s])})")
    c = (df_setor[df_setor.Serviço == s][['st_codigo','Data de abertura']]
         .groupby('st_codigo')
         .count()
         .rename(columns={'Data de abertura':'Total de Atendimentos'}))
    r = df_setores.merge(c, on='st_codigo')
    r.to_file('resultados/Atendimentos-156-2019.gpkg', layer=f'Setores por serviço - {s}', driver='GPKG')

Reclamações sobre coleta de lixo domiciliar: (3719)
Denúncia de má conduta dos funcionários de limpeza: (243)
Reclamações sobre coleta seletiva: (469)
Reclamações sobre coleta de resíduos de serviços de saúde: (242)
Denúncia de má conduta dos funcionários (motoristas /coletores): (375)
Remoção de propaganda irregular: (115)


In [155]:
zipfile = "zip:///home/fernando/dev/geo-sp156/arquivos/SIRGAS_SHP_quadraMDSF.zip!SIRGAS_SHP_quadraMDSF/SIRGAS_SHP_quadraMDSF.shp"
df_quadras = gpd.read_file(zipfile)

In [157]:
df_quadras.head(1)

,qd_id_orig,qd_setor,qd_fiscal,qd_tipo,qd_subqua,qd_area,qd_situac,qd_id,qd_tx_tipo,geometry
0,10346990,153,029,F,001,4859.1528512042,1,129531,FISCAL,"POLYGON ((347209.277 7387175.531, 347202.470 7..."


In [159]:
df_quadra['qd_setor'] = df_quadra.Setor.astype(int).apply(lambda x: '{:03d}'.format(x))
df_quadra['qd_fiscal'] = df_quadra.Quadra.astype(int).apply(lambda x: '{:03d}'.format(x))

<ipython-input-159-c582b7a93ab0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_quadra['qd_setor'] = df_quadra.Setor.astype(int).apply(lambda x: '{:03d}'.format(x))
<ipython-input-159-c582b7a93ab0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_quadra['qd_fiscal'] = df_quadra.Quadra.astype(int).apply(lambda x: '{:03d}'.format(x))


In [160]:
df_quadra.head(1)

,Data de abertura,Canal,Tema,Assunto,Serviço,Logradouro,Número,CEP,Subprefeitura,Distrito,Setor,Quadra,Latitude,Longitude,Data do parecer,Status da solicitação,Órgão,qd_setor,qd_fiscal
98,2019-01-01 08:51:30,CENTRAL TELEFÔNICA,Animais,Registro de animais - RGA,Animal encontrado com registro geral do animal...,Rua Líra Cearense,NaN,NaN,CAMPO LIMPO,CAMPO LIMPO,168.0,321.0,NaN,NaN,2019-01-16 09:23:46,FINALIZADA,COVISA-DVZ,168,321


In [168]:
for s in df_quadra.Serviço.unique():
    print(f"{s}: ({len(df_quadra[df_quadra.Serviço == s])})")
    c = (df_quadra[df_quadra.Serviço == s][['qd_setor', 'qd_fiscal','Data de abertura']]
         .groupby(['qd_setor', 'qd_fiscal'])
         .count()
         .rename(columns={'Data de abertura':'Total de Atendimentos'}))
    r = df_quadras.merge(c, on=['qd_setor', 'qd_fiscal'])
    r.to_file('resultados/Atendimentos-156-2019.gpkg', layer=f'Quadras por serviço - {s}', driver='GPKG')

Animal encontrado com registro geral do animal (RGA): (79)
Vistoria de colmeia/vespeiro instalado: (5734)
Avaliação de local com pernilongo/mosquito: (1443)
Avaliação de animal agressor em vias públicas: (544)
Avaliação de animal em sofrimento sem proprietário para eutanásia (morte sem dor): (939)
Remover animal de grande porte solto: (166)
Reclamação de local com morcego: (543)
Implantação de coleta porta-porta: (161)


## TODO Trantando pontos fora dos limites do município de São Paulo

Existem casos em que os pontos estão localizados fora dos limites do município de São Paulo e para finalidade de estudo vamos quantifica-los e discriminá-los para estudos posteriores.

In [118]:
# msp = geopandas.read_file('arquivos/SP.shp')

In [169]:
# gdf_sp = geopandas.sjoin(gdf, msp, how='left', op='within')

In [170]:
# gdf_sp[gdf_sp.municipio.isna()].to_file("resultados/fora_dos_limites.gpkg", layer='fora_dos_limites', driver="GPKG")